# Hyperparameter Tuning with Optuna

In this hands-on demo, you will learn how to leverage **Optuna**, a powerful optimization library, for efficient model tuning. We'll guide you through the process of performing **hyperparameter optimization**, demonstrating how to define the search space, objective function, and algorithm selection. Throughout the demo, you will utilize _MLflow_ to seamlessly track the model tuning process, capturing essential information such as hyperparameters, metrics, and intermediate results. By the end of the session, you will not only grasp the principles of hyperparameter optimization but also be proficient in finding the best-tuned model using various methods such as the **MLflow API** and **MLflow UI**.

By integrating Optuna and MLflow, you can efficiently optimize hyperparameters and maintain comprehensive records of your machine learning experiments, facilitating reproducibility and collaborative research.

## Learning Objectives

**By the end of this demo, you will be able to:**

- Perform hyperparameter optimization using Optuna.
- Track the model tuning process with MLflow.
- Query previous runs from an experiment using the `MLflowClient`.
- Review an MLflow Experiment for visualizing results and selecting the best run.
- Read in the best model, make a prediction, and register the model to Unity Catalog.